In [ ]:
!pip install byaldi claudette
!sudo apt-get install -y poppler-utils
!pip install -q git+https://github.com/huggingface/transformers.git qwen-vl-utils flash-attn optimum auto-gptq bitsandbytes

In [ ]:
from google.colab import userdata


In [ ]:
import base64
import os
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')# to download the ColPali model
# os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
from byaldi import RAGMultiModalModel
from claudette import *

In [ ]:
RAG = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2", verbose=1)

In [ ]:
!wget https://arxiv.org/pdf/1706.03762
!mkdir docs
!mv 1706.03762 docs/attention.pdf

In [ ]:
RAG.index(
    input_path="./docs/attention.pdf",
    index_name="attention",
    store_collection_with_index=True, # set this to false if you don't want to store the base64 representation
    overwrite=True
)

In [ ]:
RAG.index

In [ ]:
query = "What's the BLEU score for the transformer base model?"

results = RAG.search(query, k=1)

results

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16).cuda().eval()

In [ ]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", trust_remote_code=True)

In [ ]:
from pdf2image import convert_from_path

images = convert_from_path("./docs/attention.pdf")

In [ ]:
images[7]

In [ ]:
image_index = results[0]["page_num"] - 1
image_index

In [ ]:
query = "What's the BLEU score for the transformer base model?"

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": images[image_index],
            },
            {"type": "text", "text": query},
        ],
    }
]

In [ ]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [ ]:
text

In [ ]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [ ]:
inputs

In [ ]:
!export 'PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True'

In [ ]:
generated_ids = model.generate(**inputs, max_new_tokens=50)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [ ]:
print(output_text)